In [1]:
import sys 
sys.path.append('/Users/justinvhuang/Desktop/CSE-6242-Group-Project')

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from app.utils.textpreprocessing import TextPreprocessor
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import evaluate
rouge = evaluate.load('rouge')
textprepo = TextPreprocessor()
import yaml

# Load API key from config.yaml
with open("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/app/config.yaml", "r") as file:
    config = yaml.safe_load(file)

api_key = config["api_key"]

import google.generativeai as genai
userdata = {"GOOGLE_API_KEY": api_key}
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
encode_kwargs = {"normalize_embeddings": True}
embedding_function = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={"device": "cpu"},
    encode_kwargs=encode_kwargs,
)

db_faiss = FAISS.load_local("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/app/faiss_anime_index_v3", embeddings = embedding_function)
retriever = db_faiss.as_retriever(search_kwargs={"k": 50})

In [5]:

def filter_tokens(metadata):
        metadata_tokens = metadata.get("tokens", [])
        metadata_studio = metadata.get("studio", [])
        metadata_producer = metadata.get("producer", [])
        metadata_licensors = metadata.get("licensors", [])
        #metadata_cf_recs = metadata.get("cf_recs", [])
        #metadata_pop_recs = metadata.get("pop_recs", [])
        metadata_genre = metadata.get("genre", [])
        return any(token in metadata_tokens for token in query_token) or metadata["score"] > 5.0 or any(token in metadata_studio for token in query_token) or any(token in metadata_producer for token in query_token) or any(token in metadata_licensors for token in query_token) or any(token in metadata_genre for token in query_token) 

In [6]:
retriever2 = db_faiss.as_retriever(search_kwargs={"k": 50, "filter": filter_tokens})

In [10]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """
```Instructions
You are an anime afficinado who went deep weeb under your mothers basement who knows all about anime and japanese culture. 
Use the following pieces of context to summarize different animes at the end.
Use only the context to answer the question and provide a list of anime related to the answer.  
Recommend up to 3 anime and provide a one sentence description for each anime.
Always say "Thanks for using the anime recommender from Casual Correlations!" at the end of the answer.
```

Below is an example
```
Question: can you recommend me 5 animes from the 90

Summary: 

1. Neon Genesis Evangelion: A group of teenagers pilot giant robots to protect humanity from mysterious creatures known as Angels, in a series renowned for its complex characters and psychological themes.

2. Cowboy Bebop: This space-western series follows the adventures of a group of bounty hunters aboard the spaceship Bebop, with a jazzy soundtrack and stylish animation.

3. Dragon Ball Z: Goku and his friends defend Earth from powerful foes in this action-packed series that became a major phenomenon.


Thanks for using the anime recommender from Casual Correlations!

Question: what are some animes from studio ghibli

1. My Neighbor Totoro (1988): Directed by Hayao Miyazaki, this heartwarming tale follows two young sisters who encounter friendly forest spirits in rural Japan.

2. Spirited Away (2001): Also directed by Hayao Miyazaki, Spirited Away tells the story of a young girl named Chihiro who becomes trapped in a mysterious and magical world, where she must work in a bathhouse for spirits to rescue her parents.

3. Princess Mononoke (1997): Directed by Hayao Miyazaki, this epic fantasy film explores the conflict between industrialization and nature, as a young prince becomes entangled in a struggle between forest gods and human settlers.


Thanks for using the anime recommender from Casual Correlations!

Question: what are some animes similar to one piece

1. Fairy Tail: This series follows the adventures of Natsu Dragneel and his friends in the Fairy Tail guild as they take on various quests and battles in the magical land of Fiore. Like One Piece, it features a diverse cast of characters, epic battles, and a strong sense of camaraderie.

2. Naruto: Naruto follows the journey of Naruto Uzumaki, a young ninja with dreams of becoming the strongest ninja and leader of his village, the Hokage. It features a similar blend of action, humor, and heartfelt moments, as well as a focus on friendship and determination.

3. Hunter x Hunter: This series follows Gon Freecss, a young boy who aspires to become a Hunter like his father, as he embarks on various adventures and challenges in search of his father and his own identity. It shares themes of friendship, adventure, and personal growth with One Piece.


Thanks for using the anime recommender from Casual Correlations!

Question: I like the character like monkey d luffy what anime can you recommend other than one piece

1. Naruto: Naruto Uzumaki, the main character of this series, shares some similarities with Luffy. He's determined to become the strongest ninja and leader of his village, and he possesses a similar sense of optimism and loyalty to his friends.

2. Fairy Tail: Natsu Dragneel, the protagonist of Fairy Tail, is known for his adventurous spirit, boundless energy, and strong sense of loyalty to his friends, much like Luffy. The series is filled with exciting adventures and epic battles.

3. Hunter x Hunter: Gon Freecss, the main character of Hunter x Hunter, shares Luffy's sense of adventure and determination. Like Luffy, Gon is on a quest to achieve his goals and is willing to face any challenge that comes his way.

Thanks for using the anime recommender from Casual Correlations!
```

{context}

Question: {question}

Summary:"""
four_shot_prompt = PromptTemplate.from_template(template)

In [11]:
template = """
```Instructions
You are an anime afficinado who went deep weeb under your mothers basement who knows all about anime and japanese culture. 
Use the following pieces of context to summarize different animes at the end.
Use only the context to answer the question and provide a list of anime related to the answer.  
Recommend up to 3 anime and provide a one sentence description for each anime.
Always say "Thanks for using the anime recommender from Casual Correlations!" at the end of the answer.
```

Below is an example
```
Question: can you recommend me 3 animes from the 90

Summary: 

1. Neon Genesis Evangelion: A group of teenagers pilot giant robots to protect humanity from mysterious creatures known as Angels, in a series renowned for its complex characters and psychological themes.

2. Cowboy Bebop: This space-western series follows the adventures of a group of bounty hunters aboard the spaceship Bebop, with a jazzy soundtrack and stylish animation.

3. Dragon Ball Z: Goku and his friends defend Earth from powerful foes in this action-packed series that became a major phenomenon.


Thanks for using the anime recommender from Casual Correlations!

```

{context}

Question: {question}

Summary:"""
one_shot_prompt = PromptTemplate.from_template(template)

In [12]:
# create the QA chain
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0,google_api_key = GOOGLE_API_KEY,convert_system_message_to_human=True)
#zero shot gemini model
model = genai.GenerativeModel('gemini-pro')

In [13]:
rag_chain_one_shot = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | four_shot_prompt
    | llm
    | StrOutputParser()
)

rag_chain_four_shot = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | four_shot_prompt
    | llm
    | StrOutputParser()
)

rag_chain_four_shot_filter = (
    {"context": retriever2 | format_docs, "question": RunnablePassthrough()}
    | four_shot_prompt
    | llm
    | StrOutputParser()
)

In [55]:
#who is monkey d luffy?
#what are luffy's dreams?
#who does luffy look up to?
rag_chain_one_shot.invoke("who does luffy look up to?")

'1. Monkey D. Luffy is the main protagonist of the One Piece manga and anime series. He is a young pirate who wishes to succeed Gold Roger, the deceased King of the Pirates, by finding his treasure, the "One Piece".\n2. Throughout the series, Luffy gathers himself a diverse crew named the Straw Hat Pirates, including: the three-sword-wielding combatant Roronoa Zoro (sometimes referred to as Roronoa Zollo in the English manga); the thief and navigator Nami; the cowardly marksman and inventor Usopp; the amorous cook and martial artist Sanji; the anthropomorphic reindeer and doctor Tony Tony Chopper; the archaeologist Nico Robin; the cyborg shipwright Franky; the living skeleton musician Brook; and the fish-man helmsman Jimbei.\n3. Together they sail the seas in pursuit of their dreams, encountering other pirates, bounty hunters, criminal organizations, revolutionaries, secret agents and soldiers of the corrupt World Government, and various other friends and foes.'

In [56]:
luffy_one_shot = [
    "Monkey D. Luffy is the main protagonist of the anime and manga series One Piece. He is a young pirate who dreams of becoming the Pirate King by finding the legendary treasure known as the 'One Piece'.",
    "Monkey D. Luffy is the main protagonist of the One Piece manga and anime series. His dream is to succeed Gold Roger, the deceased King of the Pirates, by finding his treasure, the 'One Piece'. Throughout the series, Luffy gathers himself a diverse crew named the Straw Hat Pirates, including: the three-sword-wielding combatant Roronoa Zoro (sometimes referred to as Roronoa Zollo in the English manga); the thief and navigator Nami; the cowardly marksman and inventor Usopp; the amorous cook and martial artist Sanji; the anthropomorphic reindeer and doctor Tony Tony Chopper; the archaeologist Nico Robin; the cyborg shipwright Franky; the living skeleton musician Brook; and the fish-man helmsman Jimbei. Together they sail the seas in pursuit of their dreams, encountering other pirates, bounty hunters, criminal organizations, revolutionaries, secret agents and soldiers of the corrupt World Government, and various other friends and foes.\n\n\nThanks for using the anime recommender from Casual Correlations!",
    '1. Monkey D. Luffy is the main protagonist of the One Piece manga and anime series. He is a young pirate who wishes to succeed Gold Roger, the deceased King of the Pirates, by finding his treasure, the "One Piece".\n2. Throughout the series, Luffy gathers himself a diverse crew named the Straw Hat Pirates, including: the three-sword-wielding combatant Roronoa Zoro (sometimes referred to as Roronoa Zollo in the English manga); the thief and navigator Nami; the cowardly marksman and inventor Usopp; the amorous cook and martial artist Sanji; the anthropomorphic reindeer and doctor Tony Tony Chopper; the archaeologist Nico Robin; the cyborg shipwright Franky; the living skeleton musician Brook; and the fish-man helmsman Jimbei.\n3. Together they sail the seas in pursuit of their dreams, encountering other pirates, bounty hunters, criminal organizations, revolutionaries, secret agents and soldiers of the corrupt World Government, and various other friends and foes.'
]

In [69]:
rag_chain_four_shot.invoke("who does luffy look up to?")

'1. Monkey D. Luffy is the main protagonist of the One Piece manga and anime series. He is a young pirate who wishes to succeed Gold Roger, the deceased King of the Pirates, by finding his treasure, the "One Piece".\n2. Throughout the series, Luffy gathers himself a diverse crew named the Straw Hat Pirates, including: the three-sword-wielding combatant Roronoa Zoro (sometimes referred to as Roronoa Zollo in the English manga); the thief and navigator Nami; the cowardly marksman and inventor Usopp; the amorous cook and martial artist Sanji; the anthropomorphic reindeer and doctor Tony Tony Chopper; the archaeologist Nico Robin; the cyborg shipwright Franky; the living skeleton musician Brook; and the fish-man helmsman Jimbei. Together they sail the seas in pursuit of their dreams, encountering other pirates, bounty hunters, criminal organizations, revolutionaries, secret agents and soldiers of the corrupt World Government, and various other friends and foes.'

In [70]:
luffy_four_shot = [
    'Monkey D. Luffy is the main protagonist of the anime and manga series One Piece. He is a young pirate who dreams of becoming the Pirate King by finding the legendary treasure known as the "One Piece".',
    'Monkey D. Luffy is the series\' main protagonist, a young pirate who wishes to succeed gold roger, the deceased king of the pirates, by finding his treasure, the "one piece".',
    '1. Monkey D. Luffy is the main protagonist of the One Piece manga and anime series. He is a young pirate who wishes to succeed Gold Roger, the deceased King of the Pirates, by finding his treasure, the "One Piece".\n2. Throughout the series, Luffy gathers himself a diverse crew named the Straw Hat Pirates, including: the three-sword-wielding combatant Roronoa Zoro (sometimes referred to as Roronoa Zollo in the English manga); the thief and navigator Nami; the cowardly marksman and inventor Usopp; the amorous cook and martial artist Sanji; the anthropomorphic reindeer and doctor Tony Tony Chopper; the archaeologist Nico Robin; the cyborg shipwright Franky; the living skeleton musician Brook; and the fish-man helmsman Jimbei. Together they sail the seas in pursuit of their dreams, encountering other pirates, bounty hunters, criminal organizations, revolutionaries, secret agents and soldiers of the corrupt World Government, and various other friends and foes.'
,
]

In [61]:
model.generate_content("who does luffy look up to?").text

'Red-Haired Shanks'

In [64]:
luffy_zero_shot = [
    "Monkey D. Luffy is the main protagonist of the manga and anime series One Piece. He is a young pirate who dreams of becoming the Pirate King, the most powerful pirate in the world. Luffy is known for his cheerful personality, his determination, and his incredible strength. He is also a skilled fighter, and he has a powerful rubber body that allows him to stretch and contort himself in strange ways.\n\nLuffy is the captain of the Straw Hat Pirates, a group of young pirates who travel the world in search of adventure. The Straw Hat Pirates are a diverse group of individuals, but they all share Luffy's dream of becoming the Pirate King.\n\nLuffy is a powerful and determined pirate, but he is also a kind and compassionate person. He cares deeply for his friends, and he is always willing to fight for what he believes in. Luffy is a true hero, and he is an inspiration to many.",
    "'* To become the Pirate King\n* To find the One Piece\n* To have the freest crew on the seas\n* To meet new people and make new friends\n* To explore the world and have adventures\n* To eat lots of meat\n* To have a big party with his friends and crew'",
    "'Red-Haired Shanks'"
]

In [19]:
references_luffy = ["Monkey D. Luffy, commonly known as 'Straw Hat Luffy' or simply 'Straw Hat', is the founder, captain, and strongest combatant of the increasingly infamous and powerful Straw Hat Pirates. ",
                    "He fearlessly pursues the legendary treasure of the late Gol D. Roger in order to become the new Pirate King and reach a further, untold dream (currently known to only his crew and closest friends).",
                    "He draws his greatest inspiration from the pirate 'Red-Haired' Shanks, who (unwittingly) enabled him to eat the Devil Fruit that turned his body into rubber,sacrificed an arm to save his life, and gifted his signature straw hat as a symbol of their vow to reunite someday, after Luffy became a great pirate.",
                    ]

In [59]:
results = rouge.compute(predictions=luffy_one_shot, references=references_luffy)
print(results)

{'rouge1': 0.21951834573193796, 'rouge2': 0.062184343434343446, 'rougeL': 0.15168015491640088, 'rougeLsum': 0.15168015491640088}


In [66]:
results = rouge.compute(predictions=luffy_zero_shot, references=references_luffy)
print(results)

{'rouge1': 0.2267596702599873, 'rouge2': 0.07375647730257659, 'rougeL': 0.15299091101247095, 'rougeLsum': 0.1876136123441133}


In [71]:
results = rouge.compute(predictions=luffy_four_shot, references=references_luffy)
print(results)

{'rouge1': 0.2793715769325525, 'rouge2': 0.06923348841452291, 'rougeL': 0.16786248981370933, 'rougeLsum': 0.17111452233403454}


In [ ]:
#list 10 anime from studio ghibili? 
#can you summarize totoro to me?
#what genre is ponyo? 

In [23]:
reference_ghibli = [
    "Spirited Away, My Neighbor Totoro, Princess Mononoke, Howl's Moving Castle, Castle in the Sky, Kiki's Delivery Service, Grave of the Fireflies, Ponyo, The Wind Rises, Nausicaä of the Valley of the Wind",
    '"My Neighbor Totoro" is a heartwarming tale directed by Hayao Miyazaki. Set in post-war rural Japan, the story follows two young sisters, Satsuki and Mei, who move to the countryside with their father to be closer to their hospitalized mother. As they explore their new surroundings, they encounter playful and mysterious forest spirits, including the lovable Totoro, a giant creature who lives in the nearby forest.',
    '"Ponyo" can be categorized as a fantasy film, specifically falling under the subgenre of children\'s fantasy. It blends elements of fantasy, adventure, and a touch of romance.'
]

In [26]:
rag_chain_one_shot.invoke("what genre is ponyo")

'Ponyo is a Japanese animated fantasy film written and directed by Hayao Miyazaki.'

In [27]:
one_shot_ghibli = [
    '1. Neon Genesis Evangelion\n2. Cowboy Bebop\n3. Dragon Ball Z\n4. Fairy Tail\n5. Naruto\n6. Hunter x Hunter\n7. One Piece\n8. Bleach\n9. Fullmetal Alchemist\n10. Death Note',
    'In 1950s Japan, university professor tatsuo kusakabe and his daughters satsuki and mei (approximately ten and four years old, respectively) move into an old house close to the hospital where the girls\' mother, yasuko, is recovering from a long-term illness. the house is inhabited by small, dark, dust-like house spirits called susuwatari, that can be seen when moving from bright places to dark ones. the susuwatari leave to find another empty house. mei discovers two small spirits that lead her into the hollow of a large camphor tree. she befriends a larger spirit, which identifies itself using a series of roars that she interprets as "totoro". mei thinks totoro is the troll from her illustrated book three billy goats gruff. she falls asleep atop totoro but when satsuki finds her, she is on the ground. despite many attempts, mei cannot show her family totoro\'s tree. tatsuo comforts her, saying totoro will reveal himself when he wants to.\n\nthe girls wait for tatsuo\'s bus, which is late. mei falls asleep on satsuki\'s back and totoro appears beside them, allowing satsuki to see him for the first time. totoro has only a leaf on his head for protection against the rain, prompting satsuki to offer her umbrella to him. delighted, he gives her a bundle of nuts and seeds in return. a giant, bus-shaped cat arrives; totoro boards it and leaves. a few days after planting the seeds, the girls awaken at midnight to find totoro and his fellow spirits engaged in a ceremonial dance around the planted seeds. they join in, causing the seeds to grow into an enormous tree. totoro takes the girls for a ride on a magical flying top. in the morning, the tree is gone but the seeds have sprouted.\nthe girls discover that a planned visit by their mother has been postponed because of a setback in her treatment. mei is upset and argues with satsuki. she leaves for the hospital to take fresh corn to their mother, but gets lost on the way. mei\'s disappearance prompts satsuki and the neighbors to search for her, thinking that mei has died. in desperation, satsuki pleads for totoro\'s help. totoro summons the catbus, which carries satsuki to mei\'s location, and the sisters reunite. the bus then takes them to the hospital, where the girls learn that their mother has been kept in the hospital by a minor cold but is otherwise recovering well. the girls secretly leave the ear of corn on the windowsill, where their parents discover it.\neventually, their mother returns home and the girls play with other children while totoro and his friends watch from afar.',
    'Ponyo is a Japanese animated fantasy film written and directed by Hayao Miyazaki.'
]

In [30]:
rag_chain_four_shot.invoke("what genre is ponyo?")

'Ponyo is a 2008 Japanese animated fantasy film written and directed by Hayao Miyazaki. It was animated by Studio Ghibli for the Nippon Television Network, Dentsu, Hakuhodo, Buena Vista Home Entertainment, Mitsubishi, and distributed by Toho. The film stars Yuria Nara, Hiroki Doi, Tomoko Yamaguchi, Kazushige Nagashima, Yūki Amami, George Tokoro, Rumi Hiiragi, Akiko Yano, Kazuko Yoshiyuki and Tomoko Naraoka. It is the eighth film Miyazaki directed for Studio Ghibli, and his tenth overall. The film tells the story of Ponyo, a goldfish who escapes from the ocean and is helped by a five-year-old human boy, Sōsuke, after she is washed ashore while trapped in a glass jar. As they bond with each other, Ponyo desires to become a human girl, against the devastating circumstances brought about by her acquisition and use of magic.\n\nThe film was originally released in Japan on July 19, 2008, by distributor Toho. It was a major commercial success, grossing over $204 million worldwide and becoming

In [31]:
four_shot_ghibli = [
    '1. Neon Genesis Evangelion\n2. Cowboy Bebop\n3. Dragon Ball Z\nThanks for using the anime recommender from Casual Correlations!',
    'In 1950s Japan, university professor tatsuo kusakabe and his daughters satsuki and mei (approximately ten and four years old, respectively) move into an old house close to the hospital where the girls\' mother, yasuko, is recovering from a long-term illness. the house is inhabited by small, dark, dust-like house spirits called susuwatari, that can be seen when moving from bright places to dark ones. the susuwatari leave to find another empty house. mei discovers two small spirits that lead her into the hollow of a large camphor tree. she befriends a larger spirit, which identifies itself using a series of roars that she interprets as "totoro". mei thinks totoro is the troll from her illustrated book three billy goats gruff. she falls asleep atop totoro but when satsuki finds her, she is on the ground. despite many attempts, mei cannot show her family totoro\'s tree. tatsuo comforts her, saying totoro will reveal himself when he wants to.\n\nthe girls wait for tatsuo\'s bus, which is late. mei falls asleep on satsuki\'s back and totoro appears beside them, allowing satsuki to see him for the first time. totoro has only a leaf on his head for protection against the rain, prompting satsuki to offer her umbrella to him. delighted, he gives her a bundle of nuts and seeds in return. a giant, bus-shaped cat arrives; totoro boards it and leaves. a few days after planting the seeds, the girls awaken at midnight to find totoro and his fellow spirits engaged in a ceremonial dance around the planted seeds. they join in, causing the seeds to grow into an enormous tree. totoro takes the girls for a ride on a magical flying top. in the morning, the tree is gone but the seeds have sprouted.\nthe girls discover that a planned visit by their mother has been postponed because of a setback in her treatment. mei is upset and argues with satsuki. she leaves for the hospital to take fresh corn to their mother, but gets lost on the way. mei\'s disappearance prompts satsuki and the neighbors to search for her, thinking that mei has died. in desperation, satsuki pleads for totoro\'s help. totoro summons the catbus, which carries satsuki to mei\'s location, and the sisters reunite. the bus then takes them to the hospital, where the girls learn that their mother has been kept in the hospital by a minor cold but is otherwise recovering well. the girls secretly leave the ear of corn on the windowsill, where their parents discover it.\neventually, their mother returns home and the girls play with other children while totoro and his friends watch from afar.',
    'Ponyo is a 2008 Japanese animated fantasy film written and directed by Hayao Miyazaki. It was animated by Studio Ghibli for the Nippon Television Network, Dentsu, Hakuhodo, Buena Vista Home Entertainment, Mitsubishi, and distributed by Toho. The film stars Yuria Nara, Hiroki Doi, Tomoko Yamaguchi, Kazushige Nagashima, Yūki Amami, George Tokoro, Rumi Hiiragi, Akiko Yano, Kazuko Yoshiyuki and Tomoko Naraoka. It is the eighth film Miyazaki directed for Studio Ghibli, and his tenth overall. The film tells the story of Ponyo, a goldfish who escapes from the ocean and is helped by a five-year-old human boy, Sōsuke, after she is washed ashore while trapped in a glass jar. As they bond with each other, Ponyo desires to become a human girl, against the devastating circumstances brought about by her acquisition and use of magic.\n\nThe film was originally released in Japan on July 19, 2008, by distributor Toho. It was a major commercial success, grossing over $204 million worldwide and becoming the eighth-highest-grossing anime film of all time. It received critical acclaim for its uplifting themes, visual design, and simultaneous appeal towards young children and all audiences.\n\nAn English-language version of the film was released on August 14, 2009, to 927 theatres across the U.S., the widest opening for a Studio Ghibli film in the U.S. It was produced by the Kennedy/Marshall Company and released by Walt Disney Pictures. The film was a box office success, grossing over $204 million worldwide and becoming the eighth-highest-grossing anime film of all time. It received critical acclaim for its uplifting themes, visual design, and simultaneous appeal towards young children and all audiences.\n\nPonyo is a fantasy film.',
]

In [34]:
model.generate_content("what genre is ponyo").text

'Animated fantasy film'

In [35]:
zero_shot_ghibli = [
    "1. Spirited Away (2001)\n2. My Neighbor Totoro (1988)\n3. Princess Mononoke (1997)\n4. Kiki's Delivery Service (1989)\n5. Castle in the Sky (1986)\n6. Nausicaä of the Valley of the Wind (1984)\n7. Ponyo (2008)\n8. Howl's Moving Castle (2004)\n9. Porco Rosso (1992)\n10. The Cat Returns (2002)",
    '**Totoro**\n\nIn the 1988 animated film "Totoro," two young sisters, Satsuki and Mei, move to the countryside after their mother falls ill. They befriend a mischievous giant forest spirit named Totoro, who becomes their guardian and guide.\n\n**Plot Summary:**\n\n* Satsuki and Mei stumble upon Totoro in the nearby forest, a gentle and wise creature who resembles a large, white rabbit.\n* Totoro becomes their protector, helping them navigate the unfamiliar surroundings and cope with their mother\'s illness.\n* The sisters embark on magical adventures with Totoro, including a visit to the Catbus and a flight through the forest on Totoro\'s back.\n* However, the family faces a setback when Mei goes missing in the forest. Satsuki and Totoro search tirelessly for her.\n* With Totoro\'s help, Satsuki finds Mei safe and sound, and their bond with the forest spirit deepens.\n* The film ends with the sisters\' mother recovering and the family settling in happily in the countryside.\n\n**Themes:**\n\n* The power of imagination and the importance of preserving childhood wonder.\n* The connection between nature and human beings.\n* The strength of family and the resilience in the face of adversity.',
    'Animated fantasy film'
]

In [38]:
results = rouge.compute(predictions=one_shot_ghibli, references=reference_ghibli)
print(results)

{'rouge1': 0.12385400449749179, 'rouge2': 0.02615550576715625, 'rougeL': 0.10580113538079289, 'rougeLsum': 0.11224858863675678}


In [39]:
results = rouge.compute(predictions=four_shot_ghibli, references=reference_ghibli)
print(results)

{'rouge1': 0.08674600425966916, 'rouge2': 0.013334992946643434, 'rougeL': 0.06232370839137791, 'rougeLsum': 0.07089430389787259}


In [40]:
results = rouge.compute(predictions=zero_shot_ghibli, references=reference_ghibli)
print(results)

{'rouge1': 0.3545532052304414, 'rouge2': 0.20817369093231164, 'rougeL': 0.25853904420987134, 'rougeLsum': 0.32580389191786135}


In [41]:
space_anime_recs = ["""Cowboy Bebop: A stylish space western following a group of bounty hunters as they chase bounties across the galaxy. 
                    Legend of the Galactic Heroes: An epic space opera chronicling the rivalry between two brilliant military commanders in a sprawling conflict.
                    Space Dandy: A comedic anime about a quirky alien hunter and his companions as they encounter bizarre creatures in space.
                    Space Battleship Yamato 2199: A remake of a classic anime series, following the crew of a battleship on a perilous journey to save Earth.
                    Mobile Suit Gundam: Iron-Blooded Orphans: Set in a future where child soldiers fight for freedom in a solar system plagued by war and inequality."""]

In [ ]:
#can you recommend 5 space anime

In [60]:
rag_chain_one_shot.invoke("can you recommend 3 space anime")

In [43]:
space_one_shot = ['1. Neon Genesis Evangelion\n2. Cowboy Bebop\n3. Dragon Ball Z\n4. Space Battleship Yamato\n5. Space Dandy']

In [59]:
rag_chain_four_shot.invoke("can you recommend 3 space anime")

In [45]:
space_four_shot = ['1. Neon Genesis Evangelion\n2. Cowboy Bebop\n3. Dragon Ball Z\n4. Space Battleship Yamato\n5. Space Dandy']

In [55]:
model.generate_content("can you recommend 3 space anime").text

'1. **Cowboy Bebop:** A classic space western with a diverse cast of characters, a thrilling story, and a catchy soundtrack.\n2. **Planetes:** A realistic and poignant anime that explores the challenges and rewards of working in space.\n3. **Space Brothers:** An inspiring anime about two brothers who dream of becoming astronauts and work hard to achieve their goals.'

In [56]:
zero_shot = ['1. **Cowboy Bebop:** A classic space western with a diverse cast of characters, a thrilling story, and a catchy soundtrack.\n2. **Planetes:** A realistic and poignant anime that explores the challenges and rewards of working in space.\n3. **Space Brothers:** An inspiring anime about two brothers who dream of becoming astronauts and work hard to achieve their goals.']

In [48]:
results = rouge.compute(predictions=space_one_shot, references=space_anime_recs)
print(results)

{'rouge1': 0.11023622047244094, 'rouge2': 0.064, 'rougeL': 0.07874015748031496, 'rougeLsum': 0.11023622047244094}


In [49]:
results = rouge.compute(predictions=space_four_shot, references=space_anime_recs)
print(results)

{'rouge1': 0.11023622047244094, 'rouge2': 0.064, 'rougeL': 0.07874015748031496, 'rougeLsum': 0.11023622047244094}


In [51]:
results = rouge.compute(predictions=zero_shot, references=space_anime_recs)
print(results)

{'rouge1': 0.47736625514403286, 'rouge2': 0.23236514522821577, 'rougeL': 0.3045267489711935, 'rougeLsum': 0.43621399176954734}


In [ ]:
rag_chain_four_shot.invoke("can you recommend 3")